Dataset link : https://www.kaggle.com/datasets/kritikseth/fruit-and-vegetable-image-recognition

In [1]:
!pip install tensorflow
!pip install numpy
!pip install pandas


[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob

2024-07-17 17:49:27.191975: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-17 17:49:27.192476: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-17 17:49:27.194481: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-17 17:49:27.200641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 17:49:27.210359: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

# Get data

In [3]:
#create a list of dict {'image' : image, 'name' : name of fruit/vegetable}
#then convert them to dataframes

test_dict = []
train_dict = []
val_dict = []

In [4]:
# to get all .jpg files use glob, then split the path around /
# SINCE GLOB returns file path
# the last 2nd element would be the class

for image in glob.glob('archive/test/*/*.jpg'):
    temp = image.split("/")
    test_dict.append({"image" : image, "name" : temp[-2]})

for image in glob.glob('archive/train/*/*.jpg'):
    temp = image.split("/")
    train_dict.append({"image" : image, "name" : temp[-2]})

for image in glob.glob('archive/validation/*/*.jpg'):
    temp = image.split("/")
    val_dict.append({"image" : image, "name" : temp[-2]})

In [5]:
#create dataframes 
test_df = pd.DataFrame(test_dict)
train_df = pd.DataFrame(train_dict)
val_df = pd.DataFrame(val_dict)

In [6]:
#create generators
test_gen = ImageDataGenerator(rescale= 1./255)
train_gen = ImageDataGenerator(rescale= 1./255)
val_gen = ImageDataGenerator(rescale= 1./255)


#extract class names
class_names = set()
for img in glob.glob("archive/test/*/*.jpg"):
    temp = img.split("/")
    class_names.add(temp[-2])
class_names = list(class_names)


#create datasets
test_dataset = test_gen.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'image',
    y_col = 'name',
    target_size = (224, 224),
    classes = class_names,
    class_mode = "sparse",
    batch_size = 32
)

train_dataset = train_gen.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'image',
    y_col = 'name',
    target_size = (224, 224),
    classes = class_names,
    class_mode = "sparse",
    batch_size = 32
)

val_dataset = val_gen.flow_from_dataframe(
    dataframe = val_df,
    x_col = 'image',
    y_col = 'name',
    target_size = (224, 224),
    classes = class_names,
    class_mode = "sparse",
    batch_size = 32
)

Found 334 validated image filenames belonging to 36 classes.


Found 2780 validated image filenames belonging to 36 classes.
Found 334 validated image filenames belonging to 36 classes.


# Create model

In [9]:
conv_base = tf.keras.applications.MobileNetV2(
    weights = 'imagenet',
    include_top = False,
    input_shape = (224, 224, 3)
)

conv_base.trainable = False

model = tf.keras.Sequential([
    conv_base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(36, activation="softmax")
])

model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = 'adam',
    metrics = ["accuracy"]
)

In [11]:
!pip install pillow

model.fit(
    train_dataset,
    epochs = 15,
    validation_data = (val_dataset)
)


[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Epoch 1/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.5099 - loss: 1.7660 - val_accuracy: 0.8413 - val_loss: 0.5121
Epoch 2/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step - accuracy: 0.7958 - loss: 0.6584 - val_accuracy: 0.9072 - val_loss: 0.3191
Epoch 3/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.8867 - loss: 0.3312 - val_accuracy: 0.9341 - val_loss: 0.2945
Epoch 4/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.9293 - loss: 0.2447 - val_accuracy: 0.9581 - val_loss: 0.2010
Epoch 5/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 97s 1s/step - accuracy: 0.9669 - loss: 0.1137 - val_accuracy: 0.9611 - val_loss: 0.2905
Epoch 6/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.9812 - loss: 0.0650 - val_accuracy: 0.9641 - val_loss: 0.2100
Epoch 7/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 88s 986ms/step - accuracy: 0.9771 - loss: 0.0723 - val_accuracy: 0.9701 - val_loss: 0.2531
Epo

In [12]:
model.evaluate(test_dataset, verbose = 2)

/home/users/harshit.sahu/Desktop/github_pros/fruits_veg_cnn/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 - 11s - 1s/step - accuracy: 0.9701 - loss: 0.2387


[0.23866336047649384, 0.970059871673584]

# Check for an individual picture

In [26]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 1.9 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [27]:
import  cv2

In [30]:
model.save("test.keras")

In [38]:
img = cv2.imread("Image_4.jpg")

img = cv2.resize(img,(224,224))
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = img/255.0
img = np.expand_dims(img,axis=0)
class_names[np.argmax(model.predict(img))]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


'kiwi'

In [37]:
class_names

['bell pepper',
 'sweetcorn',
 'chilli pepper',
 'apple',
 'ginger',
 'lemon',
 'turnip',
 'eggplant',
 'raddish',
 'cauliflower',
 'garlic',
 'pear',
 'kiwi',
 'jalepeno',
 'carrot',
 'cucumber',
 'sweetpotato',
 'tomato',
 'lettuce',
 'pomegranate',
 'watermelon',
 'paprika',
 'mango',
 'soy beans',
 'cabbage',
 'spinach',
 'corn',
 'banana',
 'onion',
 'potato',
 'beetroot',
 'peas',
 'orange',
 'pineapple',
 'capsicum',
 'grapes']